In [1]:
from imblearn.ensemble import RUSBoostClassifier
import sklearn as sk
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,RepeatedKFold,GridSearchCV,RandomizedSearchCV ,StratifiedKFold,KFold
from sklearn.svm import LinearSVC , LinearSVR
from sklearn.metrics import mean_squared_error , f1_score ,accuracy_score, precision_score, recall_score, f1_score,matthews_corrcoef,auc,roc_auc_score,confusion_matrix,roc_curve
from sklearn.svm import SVR
from sklearn.preprocessing import scale ,StandardScaler
from sklearn.feature_selection import SelectKBest, chi2 ,f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import  make_scorer
from sklearn.ensemble import RandomForestClassifier ,RandomForestRegressor
from xgboost import XGBClassifier ,XGBRFRegressor ,XGBRegressor
from rotation_forest import RotationTreeClassifier, RotationForestClassifier
from sklearn.inspection import permutation_importance
from catboost import CatBoostClassifier,CatBoostRegressor, EShapCalcType, EFeaturesSelectionAlgorithm

import random
from sklearn.tree import DecisionTreeClassifier
from pathlib import Path
from imblearn.over_sampling import SMOTE
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,Normalizer,Binarizer
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
import time
from joblib import Parallel, delayed

#from lightgbm import LGBMClassifier,LGBMRegressor
from BorutaShap import BorutaShap, load_data
from sklearn.utils import shuffle
from catboost.utils import eval_metric
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import PredefinedSplit
from scipy import stats
from scipy.stats import randint

ModuleNotFoundError: No module named 'imblearn'

In [2]:
save_data_dir = r"C:\Users\12105\Desktop\HZ with gap\before_fs"
data_dir = r"C:\Users\12105\Desktop\HZ with gap\raw_data"
fi_dir = r"C:\Users\12105\Desktop\HZ with gap\fi"
os.chdir(data_dir)

In [3]:
def threshold(input,value):
    model = Binarizer(threshold=value)
    input = np.array(input)
    input = input.reshape(-1,1)
    output = model.transform(input)
    return output.flatten()


def Performance(ytrue,yhat,mag,region):
    tn , fp , fn , tp = confusion_matrix(ytrue,yhat).flatten()
    mcc = matthews_corrcoef(ytrue,yhat)
    f1 = f1_score(ytrue,yhat)
    acc = accuracy_score(ytrue,yhat)
    recall = recall_score(ytrue,yhat)
    precision = precision_score(ytrue,yhat)
    output = {"tn":tn,"fp":fp,"fn":fn,"tp":tp,"mcc":mcc,"f1":f1,"acc":acc,"recall":recall,"precision":precision,"mag":mag,"region":region}
    return output

scorefunction = make_scorer(matthews_corrcoef, greater_is_better=True)

def irrel_filter(Data, target):
    L=Data.shape[0]
    MI=[]
    cols=[]
    targ_norm = (target-target.min())/(target.max()-target.min())
    targ_med=targ_norm.median()
    targ_norm_bins = (targ_norm>targ_med).astype(int)
    for i in Data.columns:
        temp_norm = (Data[i]-Data[i].min())/(Data[i].max()-Data[i].min())
        med=temp_norm.median()
        temp_norm_bins = (temp_norm>med).astype(int)
        U = 2*temp_norm_bins+targ_norm_bins
        U0 = U[U==0].shape[0]
        U1 = U[U==1].shape[0]
        U2 = U[U==2].shape[0]
        U3 = U[U==3].shape[0]
        
        p_targ_0=(U0+U2)/L
        p_targ_1=(U1+U3)/L
        p_temp_0=(U0+U1)/L
        p_temp_1=(U2+U3)/L
        
        p_targ0_temp0=U0/L
        p_targ0_temp1=U2/L
        p_targ1_temp0=U1/L
        p_targ1_temp1=U3/L
        try:
            mi=p_targ0_temp0*np.log2(p_targ0_temp0/(p_targ_0*p_temp_0))+p_targ1_temp0*np.log2(p_targ1_temp0/(p_targ_1*p_temp_0))+p_targ0_temp1*np.log2(p_targ0_temp1/(p_targ_0*p_temp_1))+p_targ1_temp1*np.log2(p_targ1_temp1/(p_targ_1*p_temp_1))
        except ZeroDivisionError:
            mi=np.nan
        if np.isnan(mi)==True:
            mi=0
        if mi<0.001:
            mi=-1
        MI.append(mi)
        if mi==-1:
            cols.append(i)
    return MI, cols

def redundant_filter(Data,MI):
    L=Data.shape[0]
    RC=np.zeros((1,(Data.shape[1]-1)))
    first=np.zeros((1,(Data.shape[1]-1)))
    second=np.zeros((1,(Data.shape[1]-1)))
    cols=[]
    for i in range(Data.shape[1]-1):
        first_var=Data.iloc[:,i]
        first_norm=((first_var-first_var.min())/(first_var.max()-first_var.min()))
        med=first_norm.median()
        first_norm_bins=(first_norm>med).astype(int)
        for j in range((i+1),(Data.shape[1]-1)):
            sec_var=Data.iloc[:,j]
            sec_norm=((sec_var-sec_var.min())/(sec_var.max()-sec_var.min()))
            med=sec_norm.median()
            sec_norm_bins=(sec_norm>med).astype(int)
            U=2*sec_norm_bins+first_norm_bins
            U0 = U[U==0].shape[0]
            U1 = U[U==1].shape[0]
            U2 = U[U==2].shape[0]
            U3 = U[U==3].shape[0]
            
            p_first_0=(U0+U2)/L
            p_first_1=(U1+U3)/L
            p_sec_0=(U0+U1)/L
            p_sec_1=(U2+U3)/L

            p_first0_sec0=U0/L
            p_first0_sec1=U2/L
            p_first1_sec0=U1/L
            p_first1_sec1=U3/L
            try:
                Mut_Inf_temp=p_first0_sec0*np.log2(p_first0_sec0/(p_first_0*p_sec_0))+p_first1_sec0*np.log2(p_first1_sec0/(p_first_1*p_sec_0))+p_first0_sec1*np.log2(p_first0_sec1/(p_first_0*p_sec_1))+p_first1_sec1*np.log2(p_first1_sec1/(p_first_1*p_sec_1))
            except ZeroDivisionError:
                Mut_Inf_temp=np.nan
            if np.isnan(Mut_Inf_temp)==True:
                Mut_Inf_temp=0
            if Mut_Inf_temp>RC[0, i]:
                RC[0, i]=Mut_Inf_temp
                first[0, i]=i
                second[0, i]=j
        if RC[0, i]>0.2:
            if MI[(first[0, i]).astype(int)]>MI[(second[0, i]).astype(int)]:
                drop_ind=second[0, i]
            else:
                drop_ind=first[0, i]
        cols.append(Data.columns[drop_ind.astype(int)])
    cols = list(set(cols))
    return cols, RC




def model_result(model,geodata,thr_mag,regionname):
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    label = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,label,test_size=0.3,shuffle=False)
    model.fit(X_train,y_train)
    yhat = model.predict(X_test)
    result = Performance(y_test,yhat,mag=thr_mag,region=regionname)
    print(result)
    return result



def model_result2(model,geodata,thr_mag,regionname):
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    label = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,label,test_size=0.3,shuffle=False)
    X_train ,y_train= X_train[:-25] , y_train[:-25]
    X_test,y_test = X_test.iloc[25:],y_test[25:]
    model.fit(X_train,y_train)
    yhat = model.predict(X_test)
    result = Performance(y_test,yhat,mag=thr_mag,region=regionname)
    print(result)
    return result

def model_result_mrmr(model,geodata,thr_mag,regionname):
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.25,shuffle=False)

    MI1,cols = irrel_filter(X_train,y_train)
    cols2,RC = redundant_filter(X_train,MI=MI1)
    coldrop = cols2+cols

    X_train.drop(coldrop,axis=1,inplace=True)
    X_test.drop(coldrop,axis=1,inplace=True)

    y_train = threshold(y_train,thr_mag)
    y_test = threshold(y_test,thr_mag)
    
    model.fit(X_train,y_train)
    yhat = model.predict(X_test)
    result = Performance(yhat,y_test,mag=thr_mag,region=regionname)
    print(result)
    return result


def boruta_data_creator(model,geodata,n_trials,featype = "shap",savename = "fi_.csv"):
	
	geodata.dropna(inplace=True)
	x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
	X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.25,shuffle=False)

	Feature_Selector = BorutaShap(importance_measure=featype,
								classification=False,model=model)
	Feature_Selector.fit(X=X_train, y=y_train, n_trials=n_trials, sample=False,
						train_or_test = 'test', normalize=True,
				verbose=True)
	feaset = Feature_Selector.accepted +  Feature_Selector.tentative
	Feature_Selector.results_to_csv(filename=savename)

	print(feaset)
	return geodata[feaset]


def randomsearch(model,geodata,thr_mag,regionname,para1,param_grid,modelname):
    geodata.dropna(inplace=True)

    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    y = threshold(y,thr_mag)
    X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3,shuffle=False)

    model.set_params(**para1)
    split_index = [-1 if x in X_train.index else 0 for x in x.index]
    pds = PredefinedSplit(test_fold = split_index)

    clf = RandomizedSearchCV(estimator = model,
                   cv=pds,
                   param_distributions =param_grid)
    clf.fit(x, y)

    best_parameter = clf.best_params_
    model.set_params(**best_parameter)
    
    result = model_result(model,geodata,thr_mag,regionname)

    return [result , {"best_para":best_parameter,"model_name":modelname,"regionname":regionname,"mag":thr_mag}]


def gridsearch(model,geodata,thr_mag,regionname,para1,param_grid,modelname):
    geodata.dropna(inplace=True)

    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    y = threshold(y,thr_mag)
    X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3,shuffle=False)

    model.set_params(**para1)
    split_index = [-1 if x in X_train.index else 0 for x in x.index]
    pds = PredefinedSplit(test_fold = split_index)

    clf = GridSearchCV(estimator = model,
                   cv=pds,scoring = scorefunction,
                   param_grid =param_grid)
    clf.fit(x, y)

    best_parameter = clf.best_params_
    model.set_params(**best_parameter)
    
    result = model_result(model,geodata,thr_mag,regionname)

    return [result , {"best_para":best_parameter,"model_name":modelname,"regionname":regionname,"mag":thr_mag}]


def grid2search(model,geodata,thr_mag,regionname,para1,param_grid,modelname):
    geodata.dropna(inplace=True)

    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    y = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3,shuffle=False)
    
    X_train1, X_val, y_train1, y_val = train_test_split(X_train,y_train,test_size=0.3,shuffle=False)

    model.set_params(**para1)
    split_index = [-1 if x in X_train1.index else 0 for x in X_train.index]
    pds = PredefinedSplit(test_fold = split_index)

    clf = GridSearchCV(estimator = model,
                   cv=pds,
                   param_grid= param_grid)
    clf.fit(X_train, y_train)

    best_parameter = clf.best_params_
    model.set_params(**best_parameter)
    model.set_params(**para1)
    
    result = model_result(model,geodata,thr_mag,regionname)

    return [result , {"best_para":best_parameter,"model_name":modelname,"regionname":regionname,"mag":thr_mag}]


def model_result2(model,geodata,thr_mag,regionname):
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    label = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,label,test_size=0.31,shuffle=False)
    X_train ,y_train= X_train[:-25] , y_train[:-25]   
    X_test ,y_test= X_test[25:] , y_test[25:]
    model.fit(X_train,y_train)
    yhat = model.predict(X_test)
    result = Performance(y_test,yhat,mag=thr_mag,region=regionname)
    print(result)
    return result

def grid3search(model,geodata,thr_mag,regionname,para1,param_grid,modelname):
    geodata.dropna(inplace=True)

    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    y = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.31,shuffle=False)
    X_train ,y_train= X_train[:-25] , y_train[:-25]   
    X_test ,y_test= X_test[25:] , y_test[25:]

    X_train1, X_val, y_train1, y_val = train_test_split(X_train,y_train,test_size=0.3,shuffle=False)

    model.set_params(**para1)
    split_index = [-1 if x in X_train1.index else 0 for x in X_train.index]
    pds = PredefinedSplit(test_fold = split_index)

    clf = GridSearchCV(estimator = model,
                   cv=pds,
                   param_grid= param_grid)
    clf.fit(X_train, y_train)

    best_parameter = clf.best_params_
    model.set_params(**best_parameter)
    model.set_params(**para1)
    
    result = model_result2(model,geodata,thr_mag,regionname)

    return [result , {"best_para":best_parameter,"model_name":modelname,"regionname":regionname,"mag":thr_mag}]

In [4]:
def aucperformance(ytrue,ypro,mag,region):
    yhat = threshold(ypro,0.5)
    tn , fp , fn , tp = confusion_matrix(ytrue,yhat).flatten()
    mcc = matthews_corrcoef(ytrue,yhat)
    f1 = f1_score(ytrue,yhat)
    acc = accuracy_score(ytrue,yhat)
    recall = recall_score(ytrue,yhat)
    precision = precision_score(ytrue,yhat)
    auc = roc_auc_score(ytrue, ypro)
    output = {"tn":tn,"fp":fp,"fn":fn,"tp":tp,"mcc":mcc,"f1":f1,"auc":auc,"acc":acc,"recall":recall,"precision":precision,"mag":mag,"region":region}
    return output

In [5]:
def auc_result(model,geodata,thr_mag,regionname):
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    label = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,label,test_size=0.3,shuffle=False)
    X_train ,y_train= X_train[:-50] , y_train[:-50]
    model.fit(X_train,y_train)
    ypro = model.predict_proba(X_test)[:,1]
    result1 = aucperformance(y_test,ypro,mag=thr_mag,region=regionname)
    print(result1)
    return result1 , [y_test,ypro]

In [6]:
def aucsearch(model,geodata,thr_mag,regionname,para1,param_grid,modelname):
    geodata.dropna(inplace=True)

    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    y = threshold(y,thr_mag)
    X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3,shuffle=False)

    model.set_params(**para1)
    split_index = [-1 if x in X_train.index else 0 for x in x.index]
    pds = PredefinedSplit(test_fold = split_index)

    clf = GridSearchCV(estimator = model,
                   cv=pds, scoring = 'roc_auc',
                   param_grid =param_grid)
    clf.fit(x, y)

    best_parameter = clf.best_params_
    model.set_params(**best_parameter)
    
    result = auc_result(model,geodata,thr_mag,regionname)

    return [result , {"best_para":best_parameter,"model_name":modelname,"regionname":regionname,"mag":thr_mag}]

In [7]:
os.chdir(data_dir)
chile_data = pd.read_csv("CH.csv")
chile_data.dropna(inplace=True)
sc_data = pd.read_csv("SC_k.csv")
Hindu_data = pd.read_csv("H2.csv")

In [52]:
ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\h_cat.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\h_cat.csv",index=False)

In [32]:
r15,y15 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},auto_class_weights="Balanced"),chile_data,5,"ch")
r16,y16 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},auto_class_weights="Balanced"),chile_data,5.5,"ch")
r17,y17 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},auto_class_weights="Balanced"),chile_data,6,"ch")
r18,y18 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},auto_class_weights="Balanced"),chile_data,6.5,"ch")

{'tn': 1342, 'fp': 55, 'fn': 573, 'tp': 308, 'mcc': 0.4127964481432263, 'f1': 0.495176848874598, 'auc': 0.8126137003486471, 'acc': 0.7243195785776997, 'recall': 0.3496027241770715, 'precision': 0.8484848484848485, 'mag': 5, 'region': 'ch'}
{'tn': 1724, 'fp': 98, 'fn': 162, 'tp': 294, 'mcc': 0.626459956800866, 'f1': 0.6933962264150942, 'auc': 0.8797783426733684, 'acc': 0.8858647936786654, 'recall': 0.6447368421052632, 'precision': 0.75, 'mag': 5.5, 'region': 'ch'}
{'tn': 1827, 'fp': 166, 'fn': 89, 'tp': 196, 'mcc': 0.5469762341798968, 'f1': 0.6058732612055641, 'auc': 0.8648779500180457, 'acc': 0.8880597014925373, 'recall': 0.6877192982456141, 'precision': 0.5414364640883977, 'mag': 6, 'region': 'ch'}
{'tn': 1838, 'fp': 164, 'fn': 111, 'tp': 165, 'mcc': 0.478911180093545, 'f1': 0.5454545454545454, 'auc': 0.9046153122240079, 'acc': 0.87928007023705, 'recall': 0.5978260869565217, 'precision': 0.5015197568389058, 'mag': 6.5, 'region': 'ch'}


In [58]:
r15,y15 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000}),chile_data,5,"ch")
r16,y16 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000}),chile_data,5.5,"ch")
r17,y17 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000}),chile_data,6,"ch")
r18,y18 = auc_result(CatBoostClassifier(verbose=False,**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000}),chile_data,6.5,"ch")

{'tn': 1311, 'fp': 48, 'fn': 607, 'tp': 331, 'mcc': 0.4205240265364117, 'f1': 0.5026575550493546, 'auc': 0.8151065078266818, 'acc': 0.7148454505877231, 'recall': 0.35287846481876334, 'precision': 0.8733509234828496, 'mag': 5, 'region': 'ch'}
{'tn': 1679, 'fp': 100, 'fn': 205, 'tp': 313, 'mcc': 0.5964118561055477, 'f1': 0.6723952738990333, 'auc': 0.8970431525237595, 'acc': 0.8672181105790161, 'recall': 0.6042471042471043, 'precision': 0.7578692493946732, 'mag': 5.5, 'region': 'ch'}
{'tn': 1796, 'fp': 154, 'fn': 155, 'tp': 192, 'mcc': 0.47490297519926084, 'f1': 0.554112554112554, 'auc': 0.8960260104928692, 'acc': 0.8654767087505442, 'recall': 0.553314121037464, 'precision': 0.5549132947976878, 'mag': 6, 'region': 'ch'}
{'tn': 1916, 'fp': 43, 'fn': 308, 'tp': 30, 'mcc': 0.13491848620753957, 'f1': 0.145985401459854, 'auc': 0.9137526391619923, 'acc': 0.847191989551589, 'recall': 0.08875739644970414, 'precision': 0.410958904109589, 'mag': 6.5, 'region': 'ch'}


In [34]:
ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_cat.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_cat.csv",index=False)

In [67]:
r15,y15 = auc_result(LGBMClassifier(**{'learning_rate': 0.01, 'max_depth': 4, 'num_boost_round': 500}),chile_data,5,"ch")
r16,y16 = auc_result(LGBMClassifier(**{'learning_rate': 0.01, 'max_depth': 4, 'num_boost_round': 1000}),chile_data,5.5,"ch")
r17,y17 = auc_result(LGBMClassifier(**{'learning_rate': 0.05, 'max_depth': 4, 'num_boost_round': 200}),chile_data,6,"ch")
r18,y18 = auc_result(LGBMClassifier(**{'learning_rate': 0.1, 'max_depth': 2, 'num_boost_round': 500}),chile_data,6.5,"ch")

[LightGBM] [Warning] num_iterations is set=500, num_boost_round=500 will be ignored. Current value: num_iterations=500
{'tn': 1345, 'fp': 14, 'fn': 793, 'tp': 145, 'mcc': 0.2793974782648226, 'f1': 0.2643573381950775, 'auc': 0.7774533984131691, 'acc': 0.6486721811057902, 'recall': 0.15458422174840086, 'precision': 0.9119496855345912, 'mag': 5, 'region': 'ch'}
[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000
{'tn': 1674, 'fp': 105, 'fn': 210, 'tp': 308, 'mcc': 0.582848640342089, 'f1': 0.6616541353383459, 'auc': 0.8805720319211044, 'acc': 0.8628646060078363, 'recall': 0.5945945945945946, 'precision': 0.7457627118644068, 'mag': 5.5, 'region': 'ch'}
[LightGBM] [Warning] num_iterations is set=200, num_boost_round=200 will be ignored. Current value: num_iterations=200
{'tn': 1876, 'fp': 74, 'fn': 181, 'tp': 166, 'mcc': 0.5156363376403883, 'f1': 0.565587734241908, 'auc': 0.8808357348703171, 'acc': 0.8889856334349151, 'rec

In [68]:
ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_gbm.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_gbm.csv",index=False)

In [78]:
r15,y15 = auc_result(RandomForestClassifier(**{'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 500},class_weight="balanced"),chile_data,5,"ch")
r16,y16 = auc_result(RandomForestClassifier(**{'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 500},class_weight="balanced"),chile_data,5.5,"ch")
r17,y17 = auc_result(RandomForestClassifier(**{'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500},class_weight="balanced"),chile_data,6,"ch")
r18,y18 = auc_result(RandomForestClassifier(**{'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500},class_weight="balanced"),chile_data,6.5,"ch")

ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_rf.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_rf.csv",index=False)
myauc_result

{'tn': 1232, 'fp': 165, 'fn': 326, 'tp': 555, 'mcc': 0.5361468220796355, 'f1': 0.693316677076827, 'auc': 0.7630604579133006, 'acc': 0.7844600526777875, 'recall': 0.6299659477866061, 'precision': 0.7708333333333334, 'mag': 5, 'region': 'ch'}
{'tn': 1532, 'fp': 290, 'fn': 99, 'tp': 357, 'mcc': 0.5534436554314092, 'f1': 0.6473254759746147, 'auc': 0.8377903113986943, 'acc': 0.8292361720807726, 'recall': 0.7828947368421053, 'precision': 0.5517774343122102, 'mag': 5.5, 'region': 'ch'}
{'tn': 1785, 'fp': 208, 'fn': 211, 'tp': 74, 'mcc': 0.15599027761262838, 'f1': 0.26102292768959434, 'auc': 0.8166072481756322, 'acc': 0.8160667251975418, 'recall': 0.2596491228070175, 'precision': 0.2624113475177305, 'mag': 6, 'region': 'ch'}
{'tn': 1802, 'fp': 200, 'fn': 209, 'tp': 67, 'mcc': 0.1449158430738831, 'f1': 0.24677716390423574, 'auc': 0.7998251748251748, 'acc': 0.8204565408252853, 'recall': 0.2427536231884058, 'precision': 0.250936329588015, 'mag': 6.5, 'region': 'ch'}


,tn,fp,fn,tp,mcc,f1,auc,acc,recall,precision,mag,region
0,1232,165,326,555,0.536147,0.693317,0.763060,0.784460,0.629966,0.770833,5.0,ch
1,1532,290,99,357,0.553444,0.647325,0.837790,0.829236,0.782895,0.551777,5.5,ch
2,1785,208,211,74,0.155990,0.261023,0.816607,0.816067,0.259649,0.262411,6.0,ch
3,1802,200,209,67,0.144916,0.246777,0.799825,0.820457,0.242754,0.250936,6.5,ch


In [79]:
r15,y15 = auc_result(RotationForestClassifier(**{'max_depth': 4, 'n_estimators': 300}),chile_data,5,"ch")
r16,y16 = auc_result(RotationForestClassifier(**{'max_depth': 4, 'n_estimators': 300}),chile_data,5.5,"ch")
r17,y17 = auc_result(RotationForestClassifier(**{'max_depth': 4, 'n_estimators': 300}),chile_data,6,"ch")
r18,y18 = auc_result(RotationForestClassifier(**{'max_depth': 4, 'n_estimators': 300}),chile_data,6.5,"ch")

ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_rot.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_rot.csv",index=False)
myauc_result

{'tn': 1236, 'fp': 161, 'fn': 412, 'tp': 469, 'mcc': 0.4541300315574124, 'f1': 0.6207809397749834, 'auc': 0.6851681526085166, 'acc': 0.7484635645302897, 'recall': 0.5323496027241771, 'precision': 0.7444444444444445, 'mag': 5, 'region': 'ch'}
{'tn': 1705, 'fp': 117, 'fn': 153, 'tp': 303, 'mcc': 0.6193661300158719, 'f1': 0.6917808219178083, 'auc': 0.7659803666685925, 'acc': 0.8814749780509219, 'recall': 0.6644736842105263, 'precision': 0.7214285714285714, 'mag': 5.5, 'region': 'ch'}
{'tn': 1780, 'fp': 213, 'fn': 65, 'tp': 220, 'mcc': 0.5607793444885332, 'f1': 0.6128133704735376, 'auc': 0.8410823848381617, 'acc': 0.8779631255487269, 'recall': 0.7719298245614035, 'precision': 0.5080831408775982, 'mag': 6, 'region': 'ch'}
{'tn': 1960, 'fp': 42, 'fn': 57, 'tp': 219, 'mcc': 0.7914280445610117, 'f1': 0.8156424581005586, 'auc': 0.9044126163691382, 'acc': 0.956540825285338, 'recall': 0.7934782608695652, 'precision': 0.8390804597701149, 'mag': 6.5, 'region': 'ch'}


,tn,fp,fn,tp,mcc,f1,auc,acc,recall,precision,mag,region
0,1236,161,412,469,0.454130,0.620781,0.685168,0.748464,0.532350,0.744444,5.0,ch
1,1705,117,153,303,0.619366,0.691781,0.765980,0.881475,0.664474,0.721429,5.5,ch
2,1780,213,65,220,0.560779,0.612813,0.841082,0.877963,0.771930,0.508083,6.0,ch
3,1960,42,57,219,0.791428,0.815642,0.904413,0.956541,0.793478,0.839080,6.5,ch


In [80]:
r15,y15 = auc_result(RUSBoostClassifier(**{'n_estimators': 1000}),chile_data,5,"ch")
r16,y16 = auc_result(RUSBoostClassifier(**{'n_estimators': 1000}),chile_data,5.5,"ch")
r17,y17 = auc_result(RUSBoostClassifier(**{'n_estimators': 300}),chile_data,6,"ch")
r18,y18 = auc_result(RUSBoostClassifier(**{'n_estimators': 300}),chile_data,6.5,"ch")

ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_rus.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_rus.csv",index=False)
myauc_result

{'tn': 1234, 'fp': 163, 'fn': 451, 'tp': 430, 'mcc': 0.4121966097412314, 'f1': 0.5834464043419267, 'auc': 0.7199105916115042, 'acc': 0.7304653204565408, 'recall': 0.4880817253121453, 'precision': 0.7251264755480608, 'mag': 5, 'region': 'ch'}
{'tn': 1764, 'fp': 58, 'fn': 291, 'tp': 165, 'mcc': 0.4443487755404506, 'f1': 0.4860088365243005, 'auc': 0.7420585629826487, 'acc': 0.8467954345917471, 'recall': 0.3618421052631579, 'precision': 0.7399103139013453, 'mag': 5.5, 'region': 'ch'}
{'tn': 1739, 'fp': 254, 'fn': 66, 'tp': 219, 'mcc': 0.5228149418747429, 'f1': 0.5778364116094987, 'auc': 0.828328975977324, 'acc': 0.8595258999122037, 'recall': 0.7684210526315789, 'precision': 0.4630021141649049, 'mag': 6, 'region': 'ch'}
{'tn': 1952, 'fp': 50, 'fn': 272, 'tp': 4, 'mcc': -0.022484213692196577, 'f1': 0.024242424242424246, 'auc': 0.8734209630948762, 'acc': 0.858647936786655, 'recall': 0.014492753623188406, 'precision': 0.07407407407407407, 'mag': 6.5, 'region': 'ch'}


,tn,fp,fn,tp,mcc,f1,auc,acc,recall,precision,mag,region
0,1234,163,451,430,0.412197,0.583446,0.719911,0.730465,0.488082,0.725126,5.0,ch
1,1764,58,291,165,0.444349,0.486009,0.742059,0.846795,0.361842,0.739910,5.5,ch
2,1739,254,66,219,0.522815,0.577836,0.828329,0.859526,0.768421,0.463002,6.0,ch
3,1952,50,272,4,-0.022484,0.024242,0.873421,0.858648,0.014493,0.074074,6.5,ch


In [82]:
r15,y15 = auc_result(XGBClassifier(**{'max_depth': 4, 'n_estimators': 3000}),chile_data,5,"ch")
r16,y16 = auc_result(XGBClassifier(**{'max_depth': 4, 'n_estimators': 3000}),chile_data,5.5,"ch")
r17,y17 = auc_result(XGBClassifier(**{'max_depth': 4, 'n_estimators': 500}),chile_data,6,"ch")
r18,y18 = auc_result(XGBClassifier(**{'max_depth': 4, 'n_estimators': 500}),chile_data,6.5,"ch")

ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_xg.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_xg.csv",index=False)
myauc_result

{'tn': 1380, 'fp': 17, 'fn': 563, 'tp': 318, 'mcc': 0.47960595362874064, 'f1': 0.5230263157894737, 'auc': 0.8163857690835803, 'acc': 0.7453906935908692, 'recall': 0.36095346197502837, 'precision': 0.9492537313432836, 'mag': 5, 'region': 'ch'}
{'tn': 1718, 'fp': 104, 'fn': 156, 'tp': 300, 'mcc': 0.6293925720308612, 'f1': 0.6976744186046512, 'auc': 0.8991625262387583, 'acc': 0.8858647936786654, 'recall': 0.6578947368421053, 'precision': 0.7425742574257426, 'mag': 5.5, 'region': 'ch'}
{'tn': 1911, 'fp': 82, 'fn': 151, 'tp': 134, 'mcc': 0.48450022922083924, 'f1': 0.5349301397205589, 'auc': 0.8950387760671121, 'acc': 0.8977172958735733, 'recall': 0.47017543859649125, 'precision': 0.6203703703703703, 'mag': 6, 'region': 'ch'}
{'tn': 1943, 'fp': 59, 'fn': 136, 'tp': 140, 'mcc': 0.5521498850976538, 'f1': 0.5894736842105265, 'auc': 0.9506399397703746, 'acc': 0.9143985952589991, 'recall': 0.5072463768115942, 'precision': 0.7035175879396985, 'mag': 6.5, 'region': 'ch'}


,tn,fp,fn,tp,mcc,f1,auc,acc,recall,precision,mag,region
0,1380,17,563,318,0.479606,0.523026,0.816386,0.745391,0.360953,0.949254,5.0,ch
1,1718,104,156,300,0.629393,0.697674,0.899163,0.885865,0.657895,0.742574,5.5,ch
2,1911,82,151,134,0.484500,0.534930,0.895039,0.897717,0.470175,0.620370,6.0,ch
3,1943,59,136,140,0.552150,0.589474,0.950640,0.914399,0.507246,0.703518,6.5,ch


In [10]:
def boruta_result(model,geodata,thr_mag,n_trials,regionname):
	geodata.dropna(inplace=True)
	x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
	y = threshold(y,thr_mag)
	
	Feature_Selector = BorutaShap(importance_measure="shap",
								classification=True,model=model)
	Feature_Selector.fit(X=x, y=y, n_trials=n_trials, sample=False,
						train_or_test = 'test', normalize=True,
				verbose=True)
	feaset = Feature_Selector.accepted +  Feature_Selector.tentative + ["Actual Mag"]
	os.chdir(r"C:\Users\12105\Desktop\HZ with gap\auc_fi")
	Feature_Selector.results_to_csv(filename=regionname+"_"+str(thr_mag)+"_fi")
	print(feaset)
	newgeodata = geodata[feaset]
	result = auc_result(model,newgeodata,thr_mag,regionname)
	return feaset,result

In [ ]:
a,b = boruta_result()

In [25]:
brhin = chile_data[['Energy ', 'a_lsq', 'T_Rec_lsq5.6', 'Mean_Mag ', 'T_Rec_lik5.3', 'T_Rec_lik5.8', 'NU_lsq', 'NU_mlik ', 'std_b_lik', 'T_lsq_lsq5.3', 'Z Value', 'T_Rec_lsq4.4', 'T_Rec_lsq4.5', 'T_Rec_lsq5.9', 'M_def_lsq ', 'a_lik', 'x6 ', 'std_b_lsq', 'b_lsq', 'T_Days', 'x7_lsq', 'T_Rec_lsq4.0', 'M_def_Lik ', 'T_Rec_lsq6.0', 'Beta Value', 'Actual Mag']]
r15,y15 = auc_result(CatBoostClassifier(**{'depth': 6, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False,"eval_metric":"MCC"}),brhin,5,"h")
brhin = chile_data[['T_Rec_lsq6.0', 'T_Rec_lsq4.4', 'M_def_Lik ', 'Z Value', 'a_lsq', 'T_Rec_lsq4.0', 'T_Rec_lik4.2', 'std_b_lsq', 'NU_mlik ', 'T_Rec_lik5.9', 'b_lsq', 'Mean_Mag ', 'x6 ', 'Energy ', 'T_Rec_lsq5.2', 'NU_lsq', 'a_lik', 'x7_lsq', 'M_def_lsq ', 'T_Rec_lik5.0', 'std_b_lik', 'T_Days', 'T_Rec_lik4.1', 'Actual Mag']]
r16,y16 = auc_result(CatBoostClassifier(**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False}),brhin,5.5,"h")
brhin = chile_data[['T_Rec_lsq6.0', 'T_Rec_lsq4.4', 'T_Days', 'Energy ', 'M_def_Lik ', 'T_Rec_lsq5.7', 'NU_lsq', 'std_b_lik', 'T_Rec_lik5.9', 'Z Value', 'T_Rec_lsq5.8', 'T_Rec_lik4.4', 'a_lsq', 'M_def_lsq ', 'Mean_Mag ', 'x6 ', 'Beta Value', 'T_Rec_lik4.5', 'T_Rec_lik5.5', 'b_lik', 'T_Rec_lsq4.1', 'Actual Mag']]
r17,y17 = auc_result(CatBoostClassifier(**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False}),brhin,6,"h")
brhin = chile_data[['T_Rec_lsq6.0', 'T_Rec_lsq4.4', 'T_lsq_lsq5.3', 'Beta Value', 'T_Rec_lik4.8', 'M_def_Lik ', 'T_Rec_lik4.1', 'std_b_lik', 'Z Value', 'T_Rec_lsq5.8', 'a_lsq', 'T_Rec_lsq4.0', 'b_lik', 'T_Rec_lsq5.4', 'std_b_lsq', 'T_Rec_lik5.8', 'T_Days', 'T_Rec_lik5.3', 'T_Rec_lik5.4', 'T_Rec_lsq4.5', 'T_Rec_lik4.5', 'T_Rec_lik6.0', 'NU_mlik ', 'T_Rec_lsq4.6', 'T_Rec_lik5.9', 'T_Rec_lik5.5', 'T_Rec_lik4.0', 'T_Rec_lsq4.2', 'T_Rec_lik4.7', 'T_Rec_lik5.2', 'T_Rec_lsq5.9', 'T_Rec_lik5.6', 'T_Rec_lsq5.6', 'Mean_Mag ', 'x6 ', 'T_Rec_lsq5.7', 'b_lsq', 'T_Rec_lik5.0', 'T_Rec_lsq5.1', 'T_Rec_lsq5.5', 'Energy ', 'T_Rec_lik5.1', 'x7_mlk ', 'NU_lsq', 'a_lik', 'T_Rec_lik4.9', 'M_def_lsq ', 'T_Rec_lsq4.1', 'T_Rec_lsq5.0', 'T_Rec_lik4.3', 'T_Rec_lik5.7', 'T_Rec_lik4.6', 'T_Rec_lik4.4', 'x7_lsq', 'T_Rec_lik4.2', 'T_Rec_lsq4.7', 'Actual Mag']]
r18,y18 = auc_result(CatBoostClassifier(**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False,"eval_metric":"MCC","auto_class_weights":"Balanced",}),brhin,6.5,"h")

ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_cat_fs.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_cat_fs.csv",index=False)
myauc_result

{'tn': 1360, 'fp': 37, 'fn': 535, 'tp': 346, 'mcc': 0.4769359243771937, 'f1': 0.5474683544303798, 'auc': 0.8286517972272349, 'acc': 0.7489025460930641, 'recall': 0.3927355278093076, 'precision': 0.9033942558746736, 'mag': 5, 'region': 'h'}
{'tn': 1736, 'fp': 86, 'fn': 155, 'tp': 301, 'mcc': 0.6530337962232977, 'f1': 0.7141162514827996, 'auc': 0.890742653147688, 'acc': 0.8942054433713784, 'recall': 0.6600877192982456, 'precision': 0.7777777777777778, 'mag': 5.5, 'region': 'h'}
{'tn': 1840, 'fp': 153, 'fn': 65, 'tp': 220, 'mcc': 0.6215265088271323, 'f1': 0.6686930091185409, 'auc': 0.9221080800345067, 'acc': 0.9043020193151887, 'recall': 0.7719298245614035, 'precision': 0.5898123324396782, 'mag': 6, 'region': 'h'}
{'tn': 1881, 'fp': 121, 'fn': 220, 'tp': 56, 'mcc': 0.17365079665827002, 'f1': 0.2472406181015453, 'auc': 0.9177851134372873, 'acc': 0.8503072870939421, 'recall': 0.2028985507246377, 'precision': 0.3163841807909605, 'mag': 6.5, 'region': 'h'}


,tn,fp,fn,tp,mcc,f1,auc,acc,recall,precision,mag,region
0,1360,37,535,346,0.476936,0.547468,0.828652,0.748903,0.392736,0.903394,5.0,h
1,1736,86,155,301,0.653034,0.714116,0.890743,0.894205,0.660088,0.777778,5.5,h
2,1840,153,65,220,0.621527,0.668693,0.922108,0.904302,0.771930,0.589812,6.0,h
3,1881,121,220,56,0.173651,0.247241,0.917785,0.850307,0.202899,0.316384,6.5,h


In [31]:
x,y = chile_data.iloc[:,:-1],chile_data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.25,shuffle=False)
MI1,cols = irrel_filter(X_train,y_train)
cols2,RC = redundant_filter(X_train,MI=MI1)
coldrop = cols2+cols


brhin = chile_data.drop(coldrop,axis=1)


r15,y15 = auc_result(CatBoostClassifier(**{'depth': 6, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False,"eval_metric":"MCC"}),brhin,5,"h")
r16,y16 = auc_result(CatBoostClassifier(**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False}),brhin,5.5,"h")
r17,y17 = auc_result(CatBoostClassifier(**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False}),brhin,6,"h")
r18,y18 = auc_result(CatBoostClassifier(**{'depth': 4, 'learning_rate': 0.1, 'n_estimators': 1000},**{"verbose":False,"eval_metric":"MCC","auto_class_weights":"Balanced",}),brhin,6.5,"h")
ypro_result = pd.DataFrame({"5":y15,"5.5":y16,"6":y17,"6.5":y18})
myauc_result = pd.DataFrame([r15,r16,r17,r18])
ypro_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\ypro_ytrue\ch_cat_mrmr.csv",index=False)
myauc_result.to_csv(r"C:\Users\12105\Desktop\HZ with gap\auc_result\ch_cat_mrmr.csv",index=False)
myauc_result

{'tn': 1381, 'fp': 16, 'fn': 469, 'tp': 412, 'mcc': 0.5687626949972311, 'f1': 0.6294881588999236, 'auc': 0.865004220979446, 'acc': 0.7870939420544337, 'recall': 0.4676503972758229, 'precision': 0.9626168224299065, 'mag': 5, 'region': 'h'}
{'tn': 1736, 'fp': 86, 'fn': 146, 'tp': 310, 'mcc': 0.6679510724967315, 'f1': 0.727699530516432, 'auc': 0.868026267644963, 'acc': 0.8981562774363476, 'recall': 0.6798245614035088, 'precision': 0.7828282828282829, 'mag': 5.5, 'region': 'h'}
{'tn': 1835, 'fp': 158, 'fn': 65, 'tp': 220, 'mcc': 0.6159829525047198, 'f1': 0.6636500754147813, 'auc': 0.8896682247515426, 'acc': 0.902107111501317, 'recall': 0.7719298245614035, 'precision': 0.582010582010582, 'mag': 6, 'region': 'h'}
{'tn': 1815, 'fp': 187, 'fn': 56, 'tp': 220, 'mcc': 0.5994277653793284, 'f1': 0.6442166910688141, 'auc': 0.9161382096164705, 'acc': 0.8933274802458296, 'recall': 0.7971014492753623, 'precision': 0.5405405405405406, 'mag': 6.5, 'region': 'h'}


,tn,fp,fn,tp,mcc,f1,auc,acc,recall,precision,mag,region
0,1381,16,469,412,0.568763,0.629488,0.865004,0.787094,0.467650,0.962617,5.0,h
1,1736,86,146,310,0.667951,0.727700,0.868026,0.898156,0.679825,0.782828,5.5,h
2,1835,158,65,220,0.615983,0.663650,0.889668,0.902107,0.771930,0.582011,6.0,h
3,1815,187,56,220,0.599428,0.644217,0.916138,0.893327,0.797101,0.540541,6.5,h
